### This file contains the same code as xlm-roberta-span-token.ipynb, but with a small addition in the last code part. After applying XLM-RoBERTa, attention layer is added on top of the last hidden layer, in order to enhance NEs' local context semantic representation.
#### Embedded tokens representing span-text are Queries, and the embedding tokens from the rest of the input text are Keys and Values for attention layer.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# sub1 = 'drive/My Drive/Colab Notebooks/semeval_data/subtask1.parquet'
# print(sub1)

from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/My Drive/Colab Notebooks/semeval_data/subtask1.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)

In [3]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)

In [4]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)

    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]

    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
print(df.loc[0])

lang                                                            BG
art_name                                                BG_670.txt
entity                                                       Запад
start                                                          152
end                                                            156
class1                                                  Antagonist
classes2              [Conspirator, Instigator, Foreign Adversary]
text             Опитът на колективния Запад да „обезкърви Руси...
label                                                            0
input            Опитът на колективния Запад да „обезкърви Руси...
new_start_end                                           (151, 156)
Name: 0, dtype: object


In [5]:
def addTokensToInput(row):
    inp = row['input']
    start,end = row['new_start_end']
    #print(start,end)
    start = int(start)
    end = int(end)
    token_input = inp[:start] + "[SPAN_START] " + inp[start:end] + " [SPAN_END]" + inp[end:]
    return token_input

df['span_input'] = df.apply(addTokensToInput,axis=1)

In [6]:
def upStartEnd(row):
    start,end = row['new_start_end']
    start += len("[SPAN_START] ")
    end += len("[SPAN_START] ")
    return start,end

df['new_start_end'] = df.apply(upStartEnd,axis = 1)

In [7]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['span_input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
extraTokens = {
    "additional_special_tokens": ["[SPAN_START]", "[SPAN_END]"]
}
num_added_toks = tokenizer.add_special_tokens(extraTokens)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250004, 768, padding_idx=1)

In [9]:
data = df.loc[ : , ['span_input','label','new_start_end','entity']]
data['tokenized']=data.apply(preprocess_function,axis=1)

In [10]:
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [11]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]))
    tatt.append(torch.tensor(att[i]))

2535 2535 2535


In [12]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),510)
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]:
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    apid = tids[i][slower:supper]
    apatt = tatt[i][slower:supper]
    if 0 not in pid:
        apid = torch.cat((torch.tensor([0]),apid),dim=0)
        apatt = torch.cat((torch.tensor([1]),apatt),dim=0)
    if 2 not in pid:
        apid = torch.cat((apid,torch.tensor([2])),dim=0)
        apatt = torch.cat((apatt,torch.tensor([1])),dim=0)
    sliced_ids.append(apid)
    sliced_att.append(apatt)

Min = 10000
Max = 0
ind2 = 0
for i in range(len(indexes)):
    if len(sliced_ids[i]) < Min:
        Min = len(sliced_ids[i])
        ind2 = i

    if len(sliced_ids[i]) > Max:
        Max = len(sliced_ids[i])

In [13]:
input_ids = list()
att_mask = list()
for ten,att in zip(sliced_ids,sliced_att):
    if len(ten) < 512:
        padding_length = 512 - len(ten)
        padding_tensor = torch.full((padding_length,), tokenizer.pad_token_id, dtype=ten.dtype)
        padding_tensor2 = torch.full((padding_length,), 0, dtype=att.dtype)
        ten = torch.cat((ten,padding_tensor),dim=0)
        att = torch.cat((att,padding_tensor2),dim=0)
    input_ids.append(ten)
    att_mask.append(att)
inputIds = torch.stack(input_ids)
attMask = torch.stack(att_mask)

inputIds_np = inputIds.numpy()
attMask_np = attMask.numpy()
y = data['label'].values

In [14]:
from sklearn.model_selection import train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    inputIds_np, attMask_np, y, test_size=0.2, random_state=42,
)

In [15]:
X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

In [16]:
# Create TensorDatasets
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_ids, X_train_mask, y_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y_test)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [17]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn

optimizer = AdamW(model.parameters(), lr=8e-6)
classifier = nn.Linear(model.config.hidden_size * 2, 3).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")

    for batch in train_progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        batch_size = input_ids.size(0)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, output_hidden_states=True)

        hidden_states = outputs.hidden_states[-1]

        span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
        span_end_token_id = tokenizer.convert_tokens_to_ids('[SPAN_END]')

        start_mask = (input_ids == span_start_token_id)
        end_mask = (input_ids == span_end_token_id)

        entity_representations = []

        start_indices = start_mask.nonzero(as_tuple=True)[1]
        end_indices = end_mask.nonzero(as_tuple=True)[1]

        # check that span is valid and has non-zero length
        valid_spans = (start_indices != -1) & (end_indices != -1) & (start_indices <= end_indices)

        valid_start_indices = start_indices[valid_spans]
        valid_end_indices = end_indices[valid_spans]

        # to handle spans with different lengths by padding
        max_span_length = (valid_end_indices - valid_start_indices).max().item() + 1

        # extract entity tokens for every sample in batch
        for i in range(batch_size):
            if valid_spans[i]:
                entity_tokens = hidden_states[i, valid_start_indices[i]:valid_end_indices[i]]

                # padding - to have consistent dimensions for tensors
                if entity_tokens.size(0) < max_span_length:
                    padding_length = max_span_length - entity_tokens.size(0)
                    padding = torch.zeros(padding_length, entity_tokens.size(1), device=device)
                    entity_tokens = torch.cat([entity_tokens, padding], dim=0)

                entity_representations.append(entity_tokens)


        #[batch_size, max_span_length, hidden_size]
        entity_representations = torch.stack(entity_representations, dim=0) if entity_representations else \
            torch.empty((batch_size, max_span_length, hidden_states.size(-1)), device=device)

        # Calculate NE-specific context with attention
        # Entity tokens are Q, and hidden_states are K and V
        attention_scores = torch.softmax(torch.matmul(entity_representations, hidden_states[0].T), dim=-1)
        context_vector = torch.matmul(attention_scores, hidden_states[0])

        # Pooling over the span length (mean pooling)
        context_vector_pooled = context_vector.mean(dim=1)

        representation = torch.cat([entity_representations.mean(dim=1), context_vector_pooled], dim=-1)

        logits = classifier(representation) 
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        preds = torch.argmax(logits, dim=-1)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)

        train_progress_bar.set_postfix({'loss': loss.item()})

    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")

    model.eval()
    test_loss = 0
    correct_test_predictions = 0
    total_test_predictions = 0

    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")

    with torch.no_grad():
        for batch in test_progress_bar:

            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            batch_size = input_ids.size(0)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, output_hidden_states=True)

            hidden_states = outputs.hidden_states[-1]

            span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
            span_end_token_id = tokenizer.convert_tokens_to_ids('[SPAN_END]')

            start_mask = (input_ids == span_start_token_id)
            end_mask = (input_ids == span_end_token_id)

            entity_representations = []

            start_indices = start_mask.nonzero(as_tuple=True)[1]
            end_indices = end_mask.nonzero(as_tuple=True)[1]

            valid_spans = (start_indices != -1) & (end_indices != -1) & (start_indices <= end_indices)

            valid_start_indices = start_indices[valid_spans]
            valid_end_indices = end_indices[valid_spans]

            max_span_length = (valid_end_indices - valid_start_indices).max().item() + 1 

            for i in range(batch_size):
                if valid_spans[i]:
                    entity_tokens = hidden_states[i, valid_start_indices[i]:valid_end_indices[i]]

                    if entity_tokens.size(0) < max_span_length:
                        padding_length = max_span_length - entity_tokens.size(0)
                        padding = torch.zeros(padding_length, entity_tokens.size(1), device=device)
                        entity_tokens = torch.cat([entity_tokens, padding], dim=0)

                    entity_representations.append(entity_tokens)

            entity_representations = torch.stack(entity_representations, dim=0) if entity_representations else \
                torch.empty((batch_size, max_span_length, hidden_states.size(-1)), device=device)

            attention_scores = torch.softmax(torch.matmul(entity_representations, hidden_states[0].T), dim=-1)
            context_vector = torch.matmul(attention_scores, hidden_states[0])

            context_vector_pooled = context_vector.mean(dim=1)

            representation = torch.cat([entity_representations.mean(dim=1), context_vector_pooled], dim=-1)
            logits = classifier(representation)

            loss = criterion(logits, labels)
            test_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            correct_test_predictions += (preds == labels).sum().item()
            total_test_predictions += labels.size(0)

            test_progress_bar.set_postfix({'loss': loss.item()})

    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions

    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

Training Epoch 1/10: 100%|██████████| 127/127 [03:11<00:00,  1.50s/it, loss=0.861]


Epoch 1/10
Training loss: 1.0080, Training accuracy: 0.4941


Test Epoch 1/10: 100%|██████████| 32/32 [00:13<00:00,  2.34it/s, loss=1.01]


Test loss: 0.9233, Test accuracy: 0.5266


Training Epoch 2/10: 100%|██████████| 127/127 [03:10<00:00,  1.50s/it, loss=0.801]


...

Test loss: 0.6121, Test accuracy: 0.8107


Training Epoch 10/10: 100%|██████████| 127/127 [03:10<00:00,  1.50s/it, loss=0.594]


Epoch 10/10
Training loss: 0.3057, Training accuracy: 0.9172


Test Epoch 10/10: 100%|██████████| 32/32 [00:13<00:00,  2.33it/s, loss=0.426]

Test loss: 0.5690, Test accuracy: 0.8383
